In [1]:
import pandas as pd
import numpy as np
import os
# import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display


In [2]:
import os,glob,pickle

In [3]:
def get_features(filename):
    y, sr = librosa.load(filename, duration=3, offset=0.5)
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    return mfcc


In [4]:

def load_data(test_size=0.2):
    x=[]
    y=[]
    for file in glob.glob(r"C:\Users\Saurabh\ml\btp\data\*"):
        basename = os.path.basename(file)
        label = file.split('_')[-1]
        label = label.split('.')[0]
        
        for f in glob.glob(file+"\*"):
            y.append(label.lower())
            x.append(get_features(f))
    return x,y

In [5]:
x,y=load_data()

In [6]:
x=np.array(x)
# y=y.toarray()

In [7]:
print(x.shape)

(2792, 40)


In [8]:
a=[]
b=[]

In [9]:
a=x
b=y

In [10]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
enc = LabelEncoder()
enc.fit(y)
nf=enc.transform(y)

In [11]:
print(nf)

[0 0 0 ... 5 5 5]


In [12]:
ncf = nf.reshape(-1, 1) 
ohe = OneHotEncoder(sparse=False)
ncf= ohe.fit_transform(ncf)


In [13]:
y=ncf

In [14]:
x= np.expand_dims(x, -1)
x.shape

(2792, 40, 1)

In [15]:
# from keras.models import Sequential
# from keras.layers import Dense, LSTM, Dropout

# model = Sequential([
#     LSTM(256, return_sequences=False, input_shape=(40,1)),
#     Dropout(0.2),
#     Dense(128, activation='relu'),
#     Dropout(0.2),
#     Dense(64, activation='relu'),
#     Dropout(0.2),
#     Dense(8, activation='softmax')
# ])

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

In [16]:
# md = model.fit(x, y, validation_split=0.2, epochs=10, batch_size=64)

In [42]:
x.shape

(2792, 40, 1)

In [18]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(np.array(x),y,test_size=0.2,random_state=0)

In [36]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.regularizers import l2
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
# from keras.utils import to_categorical
from keras.layers import Activation, Dense
from keras.layers import Input, Flatten, Dropout, Activation
model = Sequential()

model.add(Conv1D(256, 5,padding='same',
                 input_shape=(40,1)))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Dropout(0.2))
model.add(Activation('relu'))
#model.add(Conv1D(128, 5,padding='same',))
#model.add(Activation('relu'))
#model.add(Conv1D(128, 5,padding='same',))
#model.add(Activation('relu'))
#model.add(Dropout(0.2))
model.add(Conv1D(128, 5,padding='same',))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(8))
model.add(Activation('softmax'))
opt = keras.optimizers.RMSprop(lr=0.00001, decay=1e-6)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 40, 256)           1536      
                                                                 
 dropout_2 (Dropout)         (None, 40, 256)           0         
                                                                 
 activation_9 (Activation)   (None, 40, 256)           0         
                                                                 
 conv1d_9 (Conv1D)           (None, 40, 128)           163968    
                                                                 
 dropout_3 (Dropout)         (None, 40, 128)           0         
                                                                 
 activation_10 (Activation)  (None, 40, 128)           0         
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 5, 128)          

C:\Users\Saurabh\anaconda3\envs\tensorflow\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [37]:
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [38]:
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['acc',f1_m,precision_m, recall_m])

In [40]:
cnn=model.fit(x_train, y_train,validation_data=(x_test,y_test), batch_size=16, epochs=25)

Epoch 1/25
140/140 [==============================] - 2s 12ms/step - loss: 1.0278 - acc: 0.6176 - f1_m: 0.5824 - precision_m: 0.7549 - recall_m: 0.4816 - val_loss: 0.8007 - val_acc: 0.7692 - val_f1_m: 0.5939 - val_precision_m: 0.9366 - val_recall_m: 0.4455
Epoch 2/25
140/140 [==============================] - 2s 13ms/step - loss: 0.9707 - acc: 0.6404 - f1_m: 0.6044 - precision_m: 0.7668 - recall_m: 0.5070 - val_loss: 0.7324 - val_acc: 0.7925 - val_f1_m: 0.6531 - val_precision_m: 0.9426 - val_recall_m: 0.5099
Epoch 3/25
140/140 [==============================] - 2s 13ms/step - loss: 0.8916 - acc: 0.6619 - f1_m: 0.6428 - precision_m: 0.7902 - recall_m: 0.5480 - val_loss: 0.6782 - val_acc: 0.8140 - val_f1_m: 0.6782 - val_precision_m: 0.9495 - val_recall_m: 0.5367
Epoch 4/25
140/140 [==============================] - 2s 13ms/step - loss: 0.8272 - acc: 0.7044 - f1_m: 0.6733 - precision_m: 0.8018 - recall_m: 0.5859 - val_loss: 0.6282 - val_acc: 0.8211 - val_f1_m: 0.7247 - val_precision_m: 0.

In [41]:
loss, accuracy, f1_score, precision, recall = model.evaluate(x_test, y_test, verbose=0)
# test_accuracy=model.evaluate(x_test,y_test,verbose=1)
# print(test_accuracy[1])
# test_accuracy
print("loss: ",loss)
print("f1 score: ",f1_score)
print("precision: ",precision)
print("accuracy: ",accuracy)
print("recall: ",recall)

loss:  0.2611517608165741
f1 score:  0.9199826717376709
precision:  0.9482076168060303
accuracy:  0.9194990992546082
recall:  0.8940972089767456


In [42]:
y_pred = model.predict(x_test) 
y_pred = np.argmax(y_pred, axis = 1) 
label = np.argmax(y_test,axis = 1) 

# print(pred.shape)
from sklearn.metrics import confusion_matrix
print(confusion_matrix(label,y_pred))

[[71  0  1  0  0  1  0  1]
 [ 0 55  0  1  2 10  4  0]
 [ 5  0 69  1  0  0  0  0]
 [ 6  0  1 82  0  0  5  0]
 [ 0  0  0  0 83  0  0  0]
 [ 0  0  0  0  0 77  0  0]
 [ 0  0  0  2  1  0 44  1]
 [ 3  0  0  0  0  0  0 33]]
